In [ ]:
# This notebook tests different bits and pieces that will go into the Fizeau control loop

# created 2018 June 26 by E.S.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import numpy.ma as ma
from scipy import misc, signal
from astropy.io import fits
#% matplotlib inline

In [2]:
# set some approximate parameters of the observed grism PSF

sig = 5 # sigma of Gaussian profile in x
length_y = 200 # length in y of the psf

In [ ]:
#####################################################
# THE PART BELOW IS NEW

In [ ]:
# take FFT
# if no direction of fringes is apparent --> dial one way 10 microns, then jump back and go the other way 10 microns
# if direction of fringes is apparent --> dial OPD to make them vertical

In [5]:
from matplotlib.colors import LogNorm

In [9]:
# ersatz
center_grism = [336,817]

In [10]:
# loop over sequence of grism images to make animation of how the FFT of the grism PSF changes

'''
for f in range(6950,7049):

    image, header = fits.getdata('images/lm_180507_'+
                                 str("{:0>6d}".format(f))+
                                 '.fits',
                                 0,
                                 header=True) 

    # cut out the grism image
    img_before_padding_before_FT = image[center_grism[0]-int(0.5*length_y):center_grism[0]+int(0.5*length_y),
                                     center_grism[1]-2*sig:center_grism[1]+2*sig]
    
    
    # take FFT
    
    # w/ padding
    #padI = np.pad(img_before_padding_before_FT,int(len(img_before_padding_before_FT)),'constant',constant_values=10) 
    # no padding
    padI = img_before_padding_before_FT

    PhaseExtract = np.fft.fft2(padI)
    PhaseExtract = np.fft.fftshift(PhaseExtract)

    AmpPE = np.absolute(PhaseExtract)
    ArgPE = np.angle(PhaseExtract)

    plt.subplot(1, 3, 1)
    plt.imshow(img_before_padding_before_FT, origin="lower", cmap="gray")   
    
    plt.subplot(1, 3, 2)
    plt.imshow(AmpPE, origin="lower", cmap="gray", norm=LogNorm())
    
    #plt.subplot(1, 4, 3)
    #plt.plot(np.sum(AmpPE, axis=1))
    
    plt.subplot(1, 3, 3)
    plt.semilogy(np.sum(AmpPE[:,int(0.5*img_before_padding_before_FT.shape[1]):], axis=1)) # use right-hand side of FFT image only    
    
    #frame1 = plt.gca()
    #frame1.axes.xaxis.set_ticklabels([])
    #frame1.axes.yaxis.set_ticklabels([])
    
    #plt.tight_layout()
    
    plt.suptitle(str("{:0>6d}".format(f)))
    
    #plt.show()
    
    plt.savefig("images/psf_altair_ft_"+str("{:0>6d}".format(f))+".png", overwrite=False)
    plt.clf()
'''

'\nfor f in range(6950,7049):\n\n    image, header = fits.getdata(\'images/lm_180507_\'+\n                                 str("{:0>6d}".format(f))+\n                                 \'.fits\',\n                                 0,\n                                 header=True) \n\n    # cut out the grism image\n    img_before_padding_before_FT = image[center_grism[0]-int(0.5*length_y):center_grism[0]+int(0.5*length_y),\n                                     center_grism[1]-2*sig:center_grism[1]+2*sig]\n    \n    \n    # take FFT\n    \n    # w/ padding\n    #padI = np.pad(img_before_padding_before_FT,int(len(img_before_padding_before_FT)),\'constant\',constant_values=10) \n    # no padding\n    padI = img_before_padding_before_FT\n\n    PhaseExtract = np.fft.fft2(padI)\n    PhaseExtract = np.fft.fftshift(PhaseExtract)\n\n    AmpPE = np.absolute(PhaseExtract)\n    ArgPE = np.angle(PhaseExtract)\n\n    plt.subplot(1, 3, 1)\n    plt.imshow(img_before_padding_before_FT, origin="lower", cmap

In [11]:
testArray = []
frameArray = []
plArray = []

In [12]:
# loop over sequence of grism images to make animation of how the residuals of parts of the FFT of the grism PSF changes 

for f in range(6950,7049):

    image, header = fits.getdata('images/lm_180507_'+
                                 str("{:0>6d}".format(f))+
                                 '.fits',
                                 0,
                                 header=True) 

    # cut out the grism image
    img_before_padding_before_FT = image[center_grism[0]-int(0.5*length_y):center_grism[0]+int(0.5*length_y),
                                     center_grism[1]-2*sig:center_grism[1]+2*sig]
    
    
    # take FFT
    
    # w/ padding
    #padI = np.pad(img_before_padding_before_FT,int(len(img_before_padding_before_FT)),'constant',constant_values=10) 
    # no padding
    padI = img_before_padding_before_FT

    PhaseExtract = np.fft.fft2(padI)
    PhaseExtract = np.fft.fftshift(PhaseExtract)

    AmpPE = np.absolute(PhaseExtract)
    ArgPE = np.angle(PhaseExtract)
    
    #plt.tight_layout()
    
    plt.suptitle(str("{:0>6d}".format(f)))
    
    # testing
    to_corr = np.sum(AmpPE[:,int(0.5*img_before_padding_before_FT.shape[1]):], axis=1)
    to_corr_masked = np.copy(to_corr)
    to_corr_masked[98:102] = 0
    test_symm = signal.correlate(to_corr_masked, to_corr_masked[::-1], mode='same')
    
    leftHalf = test_symm[:int(0.5*len(test_symm))]
    rightHalf = test_symm[int(0.5*len(test_symm)):]
    resid = leftHalf-rightHalf[::-1]
    plt.plot(resid)
    frameArray = np.concatenate((frameArray,[f]))
    testArray = np.concatenate((testArray,[np.median(resid)]))
    plArray = np.concatenate((plArray,[int(header['SPCTRANS'])]))
    #plt.show()
    
    #plt.savefig("images/psf_altair_corr2_"+str("{:0>6d}".format(f))+".png", overwrite=False)
    plt.clf()

KeyboardInterrupt: 

In [123]:
coeffs = np.polyfit(plArray,testArray, 2)

In [124]:
plt.scatter(plArray,testArray)
plt.plot(plArray, coeffs[2] + coeffs[1]*plArray+coeffs[0]*plArray**2)
plt.title('FFT residuals by SPCTRANS position')
plt.xlabel('SPCTRANS')
plt.ylabel('Median of Residuals Between Top and Bottom Halves of FFT')
plt.show()